Welcome to week 5 of the Noisebridge Python class! ([Noisebridge Wiki](https://www.noisebridge.net/wiki/PyClass) | [Github](https://github.com/audiodude/PythonClass))

In this lesson, we will learn about Object Oriented Programming and how classes and objects work in Python. We will also discuss decorators.

You will learn:

* How to define a class, and instantiate an object of that class
* What a constructor is in OOP
* Proper uses of the Python `self` variable
* How object properties are stored and calculated
* Basic inheritance/class hierarchies
* How to use and define decorators

Let's go!

An **object** is a data structure that encapsulates data (**instance variables**) and functions that operate on that data (**methods**). An object is created by **instantiating** a **class**, where the class can be thought of as the template for the object.

Let's look at a code example that we will refer to throughout this lesson. First let's load our schema, as we did in the previous lesson.

In [ ]:
import sqlite3

db = sqlite3.connect('db.sqlite3')
cursor = db.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS links (
  id INTEGER NOT NULL PRIMARY KEY,
  name VARCHAR(255),
  url TEXT,
  created_at TIMESTAMP,
  upvotes INTEGER DEFAULT 0,
  downvotes INTEGER DEFAULT 0
);
''')
cursor.close()
db.close()

Then, we create a class for inserting or links into the database, and retrieving our links from the database. This could be useful, for example, if we're working on a team where one person is a database expert and knows how to write SQL, but the other team members are less familiar. All of the SQL handling code for the Link is encapsulated in the class, and the application code just deals with Links without worrying how they are persisted. In practice, you would probably use a more full-featured [ORM](https://en.wikipedia.org/wiki/Object%E2%80%93relational_mapping) solution like [SQLALchemy](https://www.sqlalchemy.org/).

In [ ]:
import sqlite3
from contextlib import closing

class Link:
    DB_FILE = 'db.sqlite3'
    
    def __init__(self):
        self.db = sqlite3.connect(self.DB_FILE)
        
    def set_many(self, name, url, created_at, upvotes, downvotes):
        self.name = name
        self.url = url
        self.created_at = created_at
        self.upvotes = upvotes
        self.downvotes = downvotes
        
    def create(self, name, url, created_at, upvotes, downvotes):
        self.set_many(name, url, created_at, upvotes, downvotes)
        
        with closing(self.db.cursor()) as cursor:
            cursor.execute('''
                INSERT INTO links (name, url, created_at, upvotes, downvotes)
                VALUES (?, ?, ?, ?, ?)
            ''', (self.name, self.url, self.created_at, self.upvotes, self.downvotes))
            if cursor.rowcount == 0:
                raise ValueError('Could not insert link')
            self.id_ = cursor.lastrowid
        self.db.commit()
            
    def load(self):
        if not hasattr(self, 'id_'):
            raise ValueError('Cannot load data without setting id_ first')
        
        with closing(self.db.cursor()) as cursor:
            cursor.execute('''
                SELECT name, url, created_at, upvotes, downvotes FROM links
                WHERE id = ?
            ''', (self.id_,))
            data = cursor.fetchone()
            self.set_many(*data)

            
# Application code
google = Link()
google.create('Google', 'https://google.com', '2023-10-23', 100, 10)
id_ = google.id_

google_copy = Link()
google_copy.id_ = id_
google_copy.load()

assert google.name == google_copy.name

The first part is where we define the `Link` class. In the following code, labelled "Application code", we use the class to **instantiate** two Link objects. The first one, which is assigned to the variable `google`, is used to create a new link in the database for Google. With the second object, `google_copy`, we load the data we just inserted and use an **assertion** to check that the two objects have the same name. In Python an `assert` statement will raise an exception if the expression on it's right does not evaluate to True. Assert statements are often used in testing.

We've already seen a few of the features of classes and objects:

* Objects are created (or "instantiated") by "calling" their class objects (`Link()`)
* Objects can have properties (or "instance variables"), such as `google.name`
* In Python, the first argument to an instance method (a method on the class that operates on an object of that class) is always `self`, which can be used to access the other properties and methods of the object.
* Properties can be added to objects, and they don't need to be defined ahead of time. We can use the `hasattr` built-in function to check if a given object has a property with the given name.

Let's consider each of these points

## Constructors

In OOP (Object Oriented Programming), a constructor is a method that is used to construct, or instantiate, a member of the class (an object). So in the above example, `Link` is the class, `google` is the object, and it is constructed using the syntax `Link()`. When a class is called in this way and an object is constructed, the special `__init__` (double underscore init) method is called. This method is the constructor, which is similar in purpose to constructors in other OOP languages.

Constructors are where you "set up" your object. In this example, our objects need a connection to the database, so we create one in the constructor and assign it to `self.db`.

Constructors can have any number of arguments, though the first one is always `self`. If, for example, we want to pass in an ID when constructing the object, we could have done:

In [ ]:
class Link2:
    DB_FILE = 'db.sqlite3'
    def __init__(self, id_):
        self.id_ = id_
        self.db = sqlite3.connect(self.DB_FILE)

In many cases, a constructor will be where the properties, or instance variables, of an object are initialized, such as we just did with `id_`.

## `self`

Python passes the object itself as the first argument to every instance method. It is almost universally implemented to assign this to a parameter named `self`, though technically the name could be anything. The `self` parameter operates very similarly to `this` in other OOP languages. It allows you to access member variables and methods of the "current" object. So for example, if I had a `Link` object named `google`, I could do:

```
google.set_many('Google', 'https://google.com', '2023-10-23', 100, 10)
```

However notice that in both the `load` method and the `create` method, `set_many` is called using `self`. One way to think about this is that when I call:

`google.create('Google', 'https://google.com', '2023-10-23', 100, 10)`

and the `create` method calls:

`self.set_many`

It has the same effect as if I had called `set_many` on the `google` object itself, as I did above. Here's another example:

In [ ]:
class Fruit:
    def __init__(self, name, price):
        self.name = name
        self.price = price
        
    def discount(self, amount=.20):
        self.price = round(self.price * (1 - amount), 2)
        
    def double_default_discount(self):
        self.discount()
        self.discount()
        
apple_1 = Fruit('apple', 2.00)
apple_1.double_default_discount()
print(apple_1.name, apple_1.price)

apple_2 = Fruit('apple2', 2.00)
apple_2.discount()
apple_2.discount()
print(apple_2.name, apple_2.price)


## Properties

We have already seen some examples of object properties in our Python code. It might help to understand properties if we refer to them by their other name: instance variables. Basically, they are variables that are "scoped" to a specific instance of an object. So we can have two different fruit objects, both with a `price` property, but the prices can be different. Changing one will not affect the other.

Also note that properties do not have to be "declared" or "defined" anywhere. This is different from other OOP languages like Java, where you have to state upfront "Objects of this class will have a property 'name' which is a String" or whatever. While we will often add properties to objects in their constructors, it is not strictly necessary. Objects don't need to have the same set of properties defined during their lifetime, and we have already seen an example of this. The `Link` objects we defined only have an `id_` property if they've been recently created or an `id_` has been set on them externally.

In fact, in Python, you can add any properties you like to any object without raising an error. Though doing so will likely make your code hard to read and debug.

In [ ]:
a_fruit = Fruit('generic', 5.00)

a_fruit.why = 1
a_fruit.would = 2
a_fruit.you = 3
a_fruit.do = 4
a_fruit.this = 5

print(a_fruit.you)

In Python, we can use `hasattr` to check if a property exists on an object, as we did above. We can also use `getattr` and `setattr` in the case that we want to get or set a property, but the name is dynamic (not known when we are writing the code).

In [ ]:
import time

assert hasattr(a_fruit, 'name')
assert getattr(a_fruit, 'price')

# This has no practical purpose, except to demonstrate that the
# property name is effectively random and we don't know it ahead
# of time.
prop = 'quantity%.0f' % time.time()
print(prop)

setattr(a_fruit, prop, 10)
print(getattr(a_fruit, prop))

## Inheritance

While it is very useful to encapsulate logic into a class/object, another useful feature of OOP is class **inheritance** or **class hierarchies**. Let's look at an example.

In [ ]:
class MultiFruit(Fruit):
    def __init__(self, name, price, quantity):
        super().__init__(name, price)
        self.quantity = quantity
        
    def per_item_price(self):
        return round(self.price / self.quantity, 2)
    
grapes = MultiFruit('grapes', 0.89, 10)
print(grapes.per_item_price())
grapes.discount()
print('Grapes cost %s, or %s per grape' % (grapes.price, grapes.per_item_price()))

Here, we've created an "is a" relationship between `MultiFruit` and `Fruit`. Any object that is a MultiFruit, "is a" Fruit (but the opposite is not true!). The `MultiFruit` is said to **inherit** the definitions of `discount` and `double_default_discount` from it's **superclass** (the class that's directly "above" it in the class hierarchy, as if we were drawing a tree diagram). Because of this inheritance, we can call `grapes.discount()` on `grapes` just like on any Fruit (because it "is a" Fruit).

Also notice the use of `super()` in the constructor. Here, we want to initialize our `MultiFruit` with a name and price, just like we do for regular `Fruit`. In this simple example, we could have just initialized those properties directly and skipped the call to the superclass constructor:

In [33]:
class MultiFruit2(Fruit):
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity

However, it's usually a good practice to try to use the superclass constructor in **subclasses**, since we might later update the `Fruit` constructor to, say, calculate ripeness on initialization, and it could introduce bugs if `MultiFruit` was being initialized in a totally separate way.

Another powerful feature of inheritance is that since a `MultiFruit` "is a" `Fruit`, you can use the former anywhere the latter is required.

In [46]:
def make_fruit_basket(fruits):
    names = [fruit.name for fruit in fruits]
    total_price = sum(fruit.price for fruit in fruits)
    return 'I made you a fruit basket with %s. It costs %s.' % (', '.join(names), total_price)

print(make_fruit_basket([apple_1, grapes]))

I made you a fruit basket with apple, grapes. It costs 1.99.


The fruit basket method just cares about the name and price, which all fruits (including multifruits) have. It doesn't matter that grapes also have a quantity. In our original example with `Link`, you could perhaps have a `InMemoryLink` class that doesn't get persisted to the database, but it could still be used anywhere you need a Link.

## `isinstance`

Sometimes, you will get passed a fruit, and you will have different logic depending on if it is a `Fruit` or a `MultiFruit`. While not exactly encouraged, you can use an `isinstance` check to make a determination.

In [37]:
assert isinstance(a_fruit, Fruit)
assert not isinstance(a_fruit, MultiFruit)
assert isinstance(grapes, Fruit)
assert isinstance(grapes, MultiFruit)

Although there are superior ways of designing your OOP programs, it's good to know that `isinstance` is there if we absolutely need it. It's far superior to checking the property `.__class__` or using the built-in `type()` directly, because it inspects class hierarchies.

In [41]:
print(type(grapes))
print(grapes.__class__)

<class '__main__.MultiFruit'>
<class '__main__.MultiFruit'>


# Decorators

Technically, a decorator is a function that is passed a function and returns a function. However, they look more like "annotations" in other languages, which is generally what they're used for. Decorators will be a fairly big part of our class on Web Apps with Flask, so let's discuss them now.

In our `MultiFruit` class, the per_item_price depends solely on the price and quantity of the object. We can use a decorator to mark it as a property, and then have the ability to access it without "calling" it (though the underlying function will still be called). Let's see that in action.

In [49]:
class MultiFruit3(Fruit):
    def __init__(self, name, price, quantity):
        super().__init__(name, price)
        self.quantity = quantity
    
    @property
    def per_item_price(self):
        return round(self.price / self.quantity, 2)
    
pears = MultiFruit3('pear', 1.39, 4)
print(pears.per_item_price)

0.35


What's going on here? The Python interpreter saw the **decorator** called `property`(the @ sign is just how the decorator is invoked) and extended the behavior of the decorated function in some way. In this case, it created a property on the object that has the same name as the function that was decorated. There are lots of built-in Python decorators for various tasks. And we can write our own!

Let's say we wanted to print a message before and after one of our Fruit methods was called. We could copy paste print statements into each method body, but that's repetitive and error-prone. What happens when we want to change the message, but forget to change it everywhere.

Instead, we can use a decorator. Let's see how our decorator would look before we implement it.

In [51]:
class Fruit3:
    def __init__(self, name, price):
        self.name = name
        self.price = price
    
    @log
    def discount(self, amount=.20):
        self.price = round(self.price * (1 - amount), 2)
    
    @log
    def double_default_discount(self):
        self.discount()
        self.discount()

Now we need to implement the `log` decorator. The syntax for defining a decorator is nothing special, recalling that a decorator is simply a function that is passed a function and returns a function. The function that is passed is the function that was decorated. The funcion we return is what will get called in its place.

In [50]:
def log(func):  # It takes the original function
    def func_with_log(*args, **kwargs):
        # This is the function that will actually
        # get called. We take in *args and **kwargs so
        # we can pass them through to the original.
        print(f'About to call {func.__name__}')
        func(*args, **kwargs)
        print(f'Done calling {func.__name__}')
    return func_with_log  # Return a function

In [54]:
f = Fruit3('banana', 0.89)
f.discount()

About to call discount
Done calling discount
